## Denoising Autoencoder 代码实现

Denoising Autoencoder（降噪自动编码器）就是在Autoencoder的基础之上，为了防止过拟合问题而对输入的数据（网络的输入层）加入噪音，使学习得到的编码器W
具有较强的鲁棒性，从而增强模型的泛化能力。Denoising Autoencoder是Bengio在08年提出的，具体内容可参考其论文：

- Extracting and composing robust features with denoising autoencoders.

### 代码实现细节
#### 构建一个DAE类

In [ ]:
class DenoisingAutoencoder():
    def __init__(self, n_input, transfer_fn... ):
        ...

        # model
        self.x = tf.placeholder(dtype=tf.float32, shape=[None, self.n_input])
        self.x_corrupted = xxx
        self.hidden = self.transfer(tf.add(tf.matmul(self.x_corrupted , self.weights['w1']), self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])

        # cost
        self.cost = .5*tf.reduce_mean(tf.pow(tf.subtract(self.reconstruction, self.x), 2))

#### 三种加噪的方式

去噪自编码器模型的输入是原始输入经某种形式的加噪过程后的退化形式，加噪过程一般分为：

- 加性高斯噪声（additive gaussian noise）

In [ ]:
self.scale = tf.placeholder(dtype=tf.float32)
self.x_corrupted = tf.add(self.x, self.scale * tf.random_normal(shape=(self.n_input, )))

- 掩模噪声（mask）

In [ ]:
self.keep_prob = tf.placeholder(dtype=tf.float32)
self.x_corrupted = tf.nn.dropout(self.x, self.keep_prob)

- 椒盐噪声

In [ ]:
def salt_and_pepper_noise(X, v):
    X_noise = X.copy()
    n_features = X.shape[1]
    mn = X.min()
    mx = X.max()

    for i, sample in enumerate(X):
        mask = np.random.randint(0, n_features, v)
        for m in mask:
            if np.random.rand() < .5:
                X_noise[i][m] = mn
            else:
                X_noise[i][m] = mx
    return X_noise

### 实践代码

In [2]:
import tensorflow as tf
import numpy as np
import input_data

mnist_width = 28
n_visible = mnist_width * mnist_width
n_hidden = 500
corruption_level = 0.3

# 输入的一张图片用28x28=784的向量表示.
X = tf.placeholder("float", [None, n_visible], name='X')

# 用于将部分输入数据置为0
mask = tf.placeholder("float", [None, n_visible], name='mask')

# create nodes for hidden variables
W_init_max = 4 * np.sqrt(6. / (n_visible + n_hidden))
W_init = tf.random_uniform(shape=[n_visible, n_hidden],
                           minval=-W_init_max,
                           maxval=W_init_max)
# 编码器
W = tf.Variable(W_init, name='W')#shape:784x500
b = tf.Variable(tf.zeros([n_hidden]), name='b')#隐含层的偏置
#解码器
W_prime = tf.transpose(W)  
b_prime = tf.Variable(tf.zeros([n_visible]), name='b_prime')


def model(X, mask, W, b, W_prime, b_prime):
    tilde_X = mask * X  # corrupted X

    Y = tf.nn.sigmoid(tf.matmul(tilde_X, W) + b)  # hidden state
    Z = tf.nn.sigmoid(tf.matmul(Y, W_prime) + b_prime)  # reconstructed input
    return Z

# build model graph
Z = model(X, mask, W, b, W_prime, b_prime)

# create cost function
cost = tf.reduce_sum(tf.pow(X - Z, 2))  # minimize squared error
train_op = tf.train.GradientDescentOptimizer(0.02).minimize(cost)  # construct an optimizer

# load MNIST data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            input_ = trX[start:end]
            mask_np = np.random.binomial(1, 1 - corruption_level, input_.shape)
            sess.run(train_op, feed_dict={X: input_, mask: mask_np})

        mask_np = np.random.binomial(1, 1 - corruption_level, teX.shape)
        print(i, sess.run(cost, feed_dict={X: teX, mask: mask_np}))

('Succesfully downloaded', 'train-images-idx3-ubyte.gz', 9912422, 'bytes.')
('Extracting', 'MNIST_data/train-images-idx3-ubyte.gz')


TypeError: only integer scalar arrays can be converted to a scalar index